In [21]:
import tensorflow as tf
from tensorflow.keras import layers, models

imagen_alto = 128
imagen_ancho = 128
canales_de_color = 3

data_augmentation = tf.keras.Sequential([
    layers.RandomFlip("horizontal", input_shape=(imagen_alto, imagen_ancho, canales_de_color)),
    layers.RandomRotation(0.1),
    layers.RandomZoom(0.1),
])

input_shape = tf.keras.Input(shape=(imagen_alto, imagen_ancho, canales_de_color))

c:\Users\achav\Documents\Osteoartritis\osteo\Lib\site-packages\keras\src\layers\preprocessing\tf_data_layer.py:18: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [22]:
# Definir la arquitectura de la CNN
model = models.Sequential([
    (input_shape),  # Capa de entrada (128x128x1)
    data_augmentation,
    layers.Conv2D(32, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.Dropout(0.5),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(2, activation='softmax')  # 2 clases: Clase A y Clase B
]) 

# Compilar el modelo
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.summary()


Model: "sequential_6"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ sequential_5 (Sequential)       │ (None, 128, 128, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_8 (Conv2D)               │ (None, 126, 126, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_4 (MaxPooling2D)  │ (None, 63, 63, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_9 (Conv2D)               │ (None, 61, 61, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 61, 61, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_3 (Flatten)             │ (None, 238144)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 64)             │    15,241,280 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 2)              │           130 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 15,260,802 (58.22 MB)

 Trainable params: 15,260,802 (58.22 MB)

 Non-trainable params: 0 (0.00 B)

In [15]:
import os
import cv2
import numpy as np
import pathlib

data_dir = pathlib.Path('dataset')

# Lista para almacenar las imágenes y etiquetas
imagenes = []
etiquetas = []

# Clases disponibles
clases = ['class_A', 'class_B']

# Recorrer las carpetas de las clases
for clase in clases:
    for plate in os.listdir(data_dir / clase ):
        for file in os.listdir(data_dir / clase / plate / 'resized_128' ):
            # Leer la imagen
            imagen = cv2.imread(str(data_dir / clase / plate / 'resized_128' / file), cv2.IMREAD_UNCHANGED)
            
            # Normalizar la imagen
            imagen = imagen / 255.0
            # Agregar la imagen a la lista
            imagenes.append(imagen)
            # Agregar la etiqueta a la lista
            etiquetas.append(clases.index(clase))
            
# Convertir las listas de imágenes y etiquetas a arrays numpy
imagenes = np.array(imagenes)
etiquetas = np.array(etiquetas)


In [23]:
from sklearn.model_selection import train_test_split

# Dividir los datos en entrenamiento y validación
imagenes_entrenamiento, imagenes_validacion, etiquetas_entrenamiento, etiquetas_validacion = train_test_split(imagenes, etiquetas, test_size=0.2, random_state=42)

# Convertir las listas de imágenes y etiquetas de entrenamiento a arrays numpy
imagenes_entrenamiento = np.array(imagenes_entrenamiento)
etiquetas_entrenamiento = np.array(etiquetas_entrenamiento)

# Convertir las listas de imágenes y etiquetas de validación a arrays numpy
imagenes_validacion = np.array(imagenes_validacion)
etiquetas_validacion = np.array(etiquetas_validacion)

In [24]:
from tensorflow.keras.callbacks import TensorBoard

num_epochs = 10
batch_size = 32

# Directorio donde se guardarán los registros para TensorBoard
# directorio_logs = "logs/"

# Callback de TensorBoard
# tensorboard_callback = TensorBoard(log_dir=directorio_logs, histogram_freq=1)

# Entrenar el modelo con el callback de TensorBoard
model.fit(imagenes_entrenamiento, etiquetas_entrenamiento,
          epochs=num_epochs,
          batch_size=batch_size,
)


Epoch 1/10
194/194 ━━━━━━━━━━━━━━━━━━━━ 68s 334ms/step - accuracy: 0.8499 - loss: 1.3863
Epoch 2/10
194/194 ━━━━━━━━━━━━━━━━━━━━ 67s 346ms/step - accuracy: 0.8762 - loss: 0.3702
Epoch 3/10
194/194 ━━━━━━━━━━━━━━━━━━━━ 62s 319ms/step - accuracy: 0.8768 - loss: 0.3685
Epoch 4/10
194/194 ━━━━━━━━━━━━━━━━━━━━ 57s 291ms/step - accuracy: 0.8671 - loss: 0.3862
Epoch 5/10
194/194 ━━━━━━━━━━━━━━━━━━━━ 57s 291ms/step - accuracy: 0.8755 - loss: 0.3585
Epoch 6/10
194/194 ━━━━━━━━━━━━━━━━━━━━ 56s 289ms/step - accuracy: 0.8800 - loss: 0.3422
Epoch 7/10
194/194 ━━━━━━━━━━━━━━━━━━━━ 56s 289ms/step - accuracy: 0.8866 - loss: 0.3335
Epoch 8/10
194/194 ━━━━━━━━━━━━━━━━━━━━ 56s 290ms/step - accuracy: 0.8770 - loss: 0.3403
Epoch 9/10
194/194 ━━━━━━━━━━━━━━━━━━━━ 56s 289ms/step - accuracy: 0.8760 - loss: 0.3314
Epoch 10/10
194/194 ━━━━━━━━━━━━━━━━━━━━ 58s 299ms/step - accuracy: 0.8789 - loss: 0.3281


In [25]:
# Evaluar el modelo
test_loss, test_acc = model.evaluate(imagenes_validacion, etiquetas_validacion)
print('Precisión en datos de prueba:', test_acc)



49/49 ━━━━━━━━━━━━━━━━━━━━ 3s 59ms/step - accuracy: 0.8615 - loss: 0.3411
Precisión en datos de prueba: 0.8631374835968018


In [27]:
## Mostrar la cantidad de parámetros del modelo

model.summary()

Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_9 (Conv2D)               │ (None, 126, 126, 32)   │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_6 (MaxPooling2D)  │ (None, 63, 63, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_10 (Conv2D)              │ (None, 61, 61, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_7 (MaxPooling2D)  │ (None, 30, 30, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_11 (Conv2D)              │ (None, 28, 28, 64)     │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_3 (Flatten)             │ (None, 50176)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 64)             │     3,211,328 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 2)              │           130 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 9,801,608 (37.39 MB)

 Trainable params: 3,267,202 (12.46 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 6,534,406 (24.93 MB)